# 第５章 顧客の退会を予測する１０本ノック

- 決定木を使って顧客が退会するかどうか（分類）を予測する
- 決定木は、わかりやすく原因を分析できるので、ビジネスの現場では頻繁に利用される、らしい。

## ノック４１

- データを読み込む
- データを整形する

In [12]:
import pandas as pd

In [13]:
customer      = pd.read_csv('../materials/part_05/customer_join.csv')
uselog_months = pd.read_csv('../materials/part_05/use_log_months.csv')

In [14]:
year_months = list(uselog_months['年月'].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months['年月'] == year_months[i]]
    tmp.rename(columns={'count': 'count_0'}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months['年月'] == year_months[i-1]]
    del tmp_before['年月']
    tmp_before.rename(columns={'count': 'count_1'}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on='customer_id', how='left')
    uselog = pd.concat([uselog, tmp], ignore_index=True)

uselog.head()

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


## ノック４２

- 退会前月の退会顧客データを作成する

In [15]:
from dateutil.relativedelta import relativedelta

In [17]:
exit_customer = customer.loc[customer['is_deleted'] == 1]
exit_customer['exit_date'] = None
exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])
for i in range(len(exit_customer)):
    exit_customer['exit_date'].iloc[i] = exit_customer['end_date'].iloc[i] - relativedelta(months=1)

exit_customer['年月'] = exit_customer['exit_date'].dt.strftime('%Y%m')
uselog['年月'] = uselog['年月'].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=['customer_id', '年月'], how='left')
print(len(uselog))
exit_uselog.head()

33851


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
exit_uselog = exit_uselog.dropna(subset=['name'])

print(len(exit_uselog))
print(len(exit_uselog['customer_id'].unique()))
exit_uselog.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


## ノック４３

- 継続顧客のデータを作成する

In [19]:
conti_customer = customer.loc[customer['is_deleted'] == 0]
conti_uselog = pd.merge(uselog, conti_customer, on=['customer_id'], how='left')
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=['name'])
print(len(conti_uselog))

33851
27422


In [20]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset='customer_id')
print(len(conti_uselog))
conti_uselog.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201902,HI886359,3,5.0,XXX,C02,F,2016-12-01,NaN,CA1,...,デイタイム,7500.0,通常,4.666667,5.0,9.0,2.0,1.0,2019-04-30,28.0
1,201806,TS037478,8,6.0,XXXX,C02,M,2018-01-01,NaN,CA1,...,デイタイム,7500.0,通常,7.333333,7.5,12.0,5.0,1.0,2019-04-30,15.0
2,201808,HD849426,9,8.0,XX,C03,M,2017-08-01,NaN,CA2,...,ナイト,6000.0,入会費半額,6.416667,6.5,9.0,5.0,1.0,2019-04-30,20.0
3,201805,TS145533,2,5.0,XXXXX,C03,M,2015-10-01,NaN,CA1,...,ナイト,6000.0,通常,4.166667,4.0,7.0,2.0,1.0,2019-04-30,42.0
4,201808,TS161670,5,4.0,XXX,C01,F,2015-09-01,NaN,CA1,...,オールタイム,10500.0,通常,5.750000,5.5,9.0,1.0,1.0,2019-04-30,43.0


In [21]:
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201902,HI886359,3,5.0,XXX,C02,F,2016-12-01,NaN,CA1,...,7500.0,通常,4.666667,5.0,9.0,2.0,1.0,2019-04-30,28.0,NaN
1,201806,TS037478,8,6.0,XXXX,C02,M,2018-01-01,NaN,CA1,...,7500.0,通常,7.333333,7.5,12.0,5.0,1.0,2019-04-30,15.0,NaN
2,201808,HD849426,9,8.0,XX,C03,M,2017-08-01,NaN,CA2,...,6000.0,入会費半額,6.416667,6.5,9.0,5.0,1.0,2019-04-30,20.0,NaN
3,201805,TS145533,2,5.0,XXXXX,C03,M,2015-10-01,NaN,CA1,...,6000.0,通常,4.166667,4.0,7.0,2.0,1.0,2019-04-30,42.0,NaN
4,201808,TS161670,5,4.0,XXX,C01,F,2015-09-01,NaN,CA1,...,10500.0,通常,5.750000,5.5,9.0,1.0,1.0,2019-04-30,43.0,NaN


## ノック４４

- 予測する月の在籍期間を作成する

In [22]:
predict_data['period'] = 0
predict_data['now_date'] = pd.to_datetime(predict_data['年月'], format='%Y%m')
predict_data['start_date'] = pd.to_datetime(predict_data['start_date'])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data['now_date'][i], predict_data['start_date'][i])
    predict_data['period'][i] = int(delta.years * 12 + delta.months)
predict_data.head()

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201902,HI886359,3,5.0,XXX,C02,F,2016-12-01,NaN,CA1,...,4.666667,5.0,9.0,2.0,1.0,2019-04-30,28.0,NaN,26,2019-02-01
1,201806,TS037478,8,6.0,XXXX,C02,M,2018-01-01,NaN,CA1,...,7.333333,7.5,12.0,5.0,1.0,2019-04-30,15.0,NaN,5,2018-06-01
2,201808,HD849426,9,8.0,XX,C03,M,2017-08-01,NaN,CA2,...,6.416667,6.5,9.0,5.0,1.0,2019-04-30,20.0,NaN,12,2018-08-01
3,201805,TS145533,2,5.0,XXXXX,C03,M,2015-10-01,NaN,CA1,...,4.166667,4.0,7.0,2.0,1.0,2019-04-30,42.0,NaN,31,2018-05-01
4,201808,TS161670,5,4.0,XXX,C01,F,2015-09-01,NaN,CA1,...,5.750000,5.5,9.0,1.0,1.0,2019-04-30,43.0,NaN,35,2018-08-01


## ノック４５

- 欠損値を除去する

In [23]:
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1               278
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [24]:
predict_data = predict_data.dropna(subset=['count_1'])
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2616
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2616
period                  0
now_date                0
dtype: int64

## ノック４６

- 文字列型の変数を処理できる形式に整形する

In [25]:
target_col = ['campaign_name', 'class_name', 'gender', 'count_1', 'routine_flg', 'period', 'is_deleted']
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,通常,デイタイム,F,5.0,1.0,26,0.0
1,通常,デイタイム,M,6.0,1.0,5,0.0
2,入会費半額,ナイト,M,8.0,1.0,12,0.0
3,通常,ナイト,M,5.0,1.0,31,0.0
4,通常,オールタイム,F,4.0,1.0,35,0.0


In [26]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,5.0,1.0,26,0.0,0,0,1,0,1,0,1,0
1,6.0,1.0,5,0.0,0,0,1,0,1,0,0,1
2,8.0,1.0,12,0.0,1,0,0,0,0,1,0,1
3,5.0,1.0,31,0.0,0,0,1,0,0,1,0,1
4,4.0,1.0,35,0.0,0,0,1,1,0,0,1,0


In [27]:
del predict_data['campaign_name_通常']
del predict_data['class_name_ナイト']
del predict_data['gender_M']
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,5.0,1.0,26,0.0,0,0,0,1,1
1,6.0,1.0,5,0.0,0,0,0,1,0
2,8.0,1.0,12,0.0,1,0,0,0,0
3,5.0,1.0,31,0.0,0,0,0,0,0
4,4.0,1.0,35,0.0,0,0,1,0,1


## ノック４７

- 決定木を用いて退会予測モデルを作成する

In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [28]:
exit  = predict_data.loc[predict_data['is_deleted'] == 1]
conti = predict_data.loc[predict_data['is_deleted'] == 0].sample(len(exit))

In [29]:
X = pd.concat([exit, conti], ignore_index=True)
y = X['is_deleted']
del X['is_deleted']

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [33]:
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0.
 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0.
 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0.
 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0.
 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0.
 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1.
 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0.
 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1.
 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1.
 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1.
 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.
 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1.

In [34]:
results_test = pd.DataFrame({'y_test': y_test, 'y_pred': y_test_pred})
results_test.head()

,y_test,y_pred
1350,0.0,0.0
371,1.0,1.0
940,1.0,1.0
297,1.0,1.0
37,1.0,1.0


## ノック４８

- 予測モデルを評価する
- 予測モデルをチューニングする

In [35]:
correct = len(results_test.loc[results_test['y_test'] == results_test['y_pred']])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.9011406844106464


In [36]:
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.982256020278834
0.9011406844106464


In [37]:
X = pd.concat([exit, conti], ignore_index=True)
y = X['is_deleted']
del X['is_deleted']

X_train, X_test, y_train, y_test = train_test_split(X, y)

model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)

print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.9372623574144486
0.908745247148289


## ノック４９

- モデルに寄与している変数を確認する

In [38]:
importance = pd.DataFrame({
    'feature_names': X.columns,
    'coefficient': model.feature_importances_,
})
importance

,feature_names,coefficient
0,count_1,0.360508
1,routine_flg,0.140073
2,period,0.496881
3,campaign_name_入会費半額,0.000000
4,campaign_name_入会費無料,0.002341
5,class_name_オールタイム,0.000198
6,class_name_デイタイム,0.000000
7,gender_F,0.000000


## ノック５０

- 顧客の退会を予測してみる

In [39]:
count_1       = 3
routine_flg   = 1
period        = 10
campaign_name = '入会費無料'
class_name    = 'オールタイム'
gender        = 'M'

In [40]:
if campaign_name == '入会費半額':
    campaign_name_list = [1, 0]
elif campaign_name == '入会費無料':
    campaign_name_list = [0, 1]
elif campaign_name == '通常':
    campaign_name_list = [0, 0]

if class_name == 'オールタイム':
    class_name_list = [1, 0]
elif class_name == 'デイタイム':
    class_name_list = [0, 1]
elif class_name == 'ナイト':
    class_name_list = [0, 0]

if gender == 'F':
    gender_list = [1]
elif gender == 'M':
    gender_list = [0]

input_data = [count_1, routine_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [41]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]
[[0.00263158 0.99736842]]
